In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import keras
import csv
import gc
import cv2

# Importing the libraries

import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
# from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
import tensorflow as tf
from tqdm import tqdm

In [ ]:
train_csv = "../input/train.csv"
test_csv = "../input/test.csv"
train_dir = "../input/train_images/"
test_dir = "../input/test_images/"
size = 224 # input image size

In [ ]:
df = pd.read_csv(train_csv)
df_test = pd.read_csv(test_csv)

In [ ]:
df_test.head()

In [ ]:
def df_train_test_split_preprocess(df):
    image_ids = df["id_code"].values.tolist()
    labels = df["diagnosis"].values.tolist()
    for i in range(len(image_ids)):
        imgname = image_ids[i]
        newname = str(imgname)+'.png'
        image_ids[i] = newname
    xtrain,xval,ytrain,yval = train_test_split(image_ids, labels, test_size = 0.1)
    df_train = pd.DataFrame({"id_code":xtrain, "diagnosis":ytrain})
    df_val = pd.DataFrame({"id_code":xval, "diagnosis":yval})
    df_train["diagnosis"] = df_train["diagnosis"].astype('str')
    df_val["diagnosis"] = df_val["diagnosis"].astype("str")
    print("Length of Training Data:",len(df_train))
    print("Length of Test Data:",len(df_val))
    return df_train, df_val

In [ ]:
df_train, df_val = df_train_test_split_preprocess(df)

Plotting the ditribution of labels

In [ ]:
no_dr= 0 
mild=0
moderate=0
severe=0
proliferative=0
for i in range(len(df)):
    if df['diagnosis'][i] == 0:
        no_dr+=1
    if df['diagnosis'][i] == 1:
        mild+=1
    if df['diagnosis'][i] == 2:
        moderate+=1
    if df['diagnosis'][i] == 3:
        severe+=1
    if df['diagnosis'][i] == 4:
        proliferative+=1
        
explode = [0.1,0,0,0,0]
labels = 'no_dr','mild','moderate','severe','proliferative'

sizes = [no_dr, mild, moderate, severe, proliferative]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode = explode, labels=labels, autopct='%1.1f%%',
       shadow=True, startangle=90)
ax1.axis('equal')

In [ ]:
sample_to_show = ['07419eddd6be.png','0124dffecf29.png']
def get_cropped_image(image):
    #img = cv2.blur(image,(2,2))
    img = image
    slice1Copy = np.uint8(img)
    canny = cv2.Canny(slice1Copy, 0,50)
    pts = np.argwhere(canny>0)
    y1, x1 = pts.min(axis=0)
    y2, x2 = pts.max(axis=0)
    cropped_img = img[y1:y2, x1:x2]
    cropped_img = cv2.resize(cropped_img,(size,size))
    cropped_img = cropped_img.astype("float32")*(1.)/255
    return np.array(cropped_img)

names =[]
samples = []
cropped_images = []
for i in sample_to_show:
    path = train_dir+str(i)
    img_ = cv2.imread(path)
    samples.append(img_)
    cropped_ = get_cropped_image(img_)
    cropped_images.append(cropped_)

fig = plt.figure(figsize=(5,5))
ax1 = fig.add_subplot(2,2,1)
ax1.title.set_text('original image')
ax1.axis("off")
plt.imshow(samples[0])
ax2 = fig.add_subplot(2,2,2)
ax2.title.set_text('croped image')
ax2.axis("off")
plt.imshow(cropped_images[0])
ax3 = fig.add_subplot(2,2,3)
ax3.title.set_text('original image')
ax3.axis('off')
plt.imshow(samples[1])
ax4 = fig.add_subplot(2,2,4)
ax4.title.set_text('cropped image')
ax4.axis('off')
plt.imshow(cropped_images[1])

In [ ]:
def sample_original_images(df, cols=4, rows=3):
    fig = plt.figure(figsize=(5*cols,4*rows))
    path = train_dir
    for i in range(cols*rows):
        rand = np.random.randint(0,len(df))
        image_name = df.loc[rand,'id_code']+".png"
        img=cv2.imread(os.path.join(path,image_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, cols, i+1)
        plt.title('image_name')
        plt.imshow(img)
        
    plt.tight_layout()
    
sample_original_images(df)
        

#cropping:


In [ ]:
def crop_image_from_gray(img, tol=7):
    if img.ndim == 2:
        mask = img>tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1 = img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2 = img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3 = img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:
def circle_crop(img):
    img = crop_image_from_gray(img)
    
    heigh, width, depth = img.shape
    largest_side = np.max((heigh, width))
    img = cv2.resize(img, (largest_side, largest_side))
    
    height, width, depth = img.shape
    
    x = int(width / 2)
    y = int(height / 2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img,(x,y), int(r), 1,thickness = -1)
    
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    return img

In [ ]:
def load_ben_color(path, sigmaX = 10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = circle_crop(image)
    image = cv2.resize(image, (size, size))
    image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)   
    return image

In [ ]:
def process_images(path, size=size):
    img = Image.open(path)
    image = load_ben_color(path, sigmaX=20)
    return image

In [ ]:
#x_train, y_train, x_val, y_val = df_train['id_code'], df_train['diagnosis'], df_val['id_code'], df_val['diagnosis']

In [ ]:
#code copied from above for sample change, only few changes made

In [ ]:
def sample_original_images(df, cols=4, rows=3):
    fig = plt.figure(figsize=(5*cols,4*rows))
    path = train_dir
    for i in range(cols*rows):
        rand = np.random.randint(0,len(df))
        image_name = df.loc[rand,'id_code']+".png"
        #img=cv2.imread(os.path.join(path,image_name))
        
        path_1=os.path.join(path,image_name)
        img = load_ben_color(path_1, sigmaX=10) #40 0.20  20  0.50
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fig.add_subplot(rows, cols, i+1)
        plt.title(image_name)
        plt.imshow(img)
        
    plt.tight_layout()
    
sample_original_images(df)

train data processing

In [ ]:
xtrain=df['id_code']+'.png'
xtest = df_test['id_code']+'.png'

In [ ]:
N = xtrain.shape[0]
X_train_ = np.empty((N,size,size,3), dtype = np.uint8)

for i, image_id in enumerate(tqdm(xtrain)):
    path_1 = os.path.join(train_dir,image_id)
    X_train_[i,:,:,:] = process_images(path_1)

#PROCESSING validation data

In [ ]:
N = xtest.shape[0]
X_test_ = np.empty((N,size,size,3), dtype=np.uint8)

for i , image_id in enumerate(tqdm(xtest)):
    path_1 = os.path.join(test_dir,image_id)
    X_test_[i,:,:,:] = process_images(path_1)

#making dummies of target variable

In [ ]:
y_train_ = pd.get_dummies(df['diagnosis']).values

print(X_train_.shape)
print(y_train_.shape)
print(X_test_.shape)

In [ ]:
# Converting our target labels into multi-labels

# y_train_multi = np.empty(y_train_.shape, dtype = y_train_.dtype)
# y_train_multi[:, 4] = y_train_[:, 4]

# for i in range(3, -1, -1):
#     y_train_multi[:, i] = np.logical_or(y_train_[:, i], y_train_multi[:, i+1])

# print("Original y_train:", y_train_.sum(axis = 0))
# print("Multilabel version:", y_train_multi.sum(axis = 0))

In [ ]:
# y_train_multi[:5]

In [ ]:
# Splitting our data into training and cross-validations sets

x_train, x_val, y_train, y_val = train_test_split(X_train_, y_train_, test_size = 0.15, random_state = 77) 

create image data generator


In [ ]:
X_train_.shape, x_train.shape
# x_val

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                            zoom_range=0.1,
                            fill_mode='constant',
                            cval=0,
                            horizontal_flip=True,
                            vertical_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
batches = train_datagen.flow(x_train, y_train, batch_size = 32, seed=2019)
val_batches = val_datagen.flow(x_val, y_val, batch_size = 32, seed=2019)

In [ ]:
# y_train

In [ ]:
# y_val

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization

In [ ]:
classifier = Sequential()
# First CONV-ReLU Layer
classifier.add(Conv2D(64, (3, 3), padding = 'same', input_shape = (size,size, 3)))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())

# Second CONV-ReLU Layer
classifier.add(Conv2D(64, (3, 3), padding = "same", input_shape = (size,size, 3)))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())

# Max Pooling with Dropout 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# classifier.add(Dropout(0.2))

# 3rd set of CONV-ReLU Layers
classifier.add(Conv2D(128, (3, 3), padding="same"))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())

# 4th Set of CONV-ReLU Layers
classifier.add(Conv2D(128, (3, 3), padding="same"))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())

# Max Pooling with Dropout 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# classifier.add(Dropout(0.2))

# 5th Set of CONV-ReLU Layers
classifier.add(Conv2D(256, (3, 3), padding="same"))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())

# 6th Set of CONV-ReLU Layers
classifier.add(Conv2D(256, (3, 3), padding="same"))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())

# Max Pooling with Dropout 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# classifier.add(Dropout(0.2))


# 5th Set of CONV-ReLU Layers
classifier.add(Conv2D(256, (3, 3), padding="same"))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())

# 6th Set of CONV-ReLU Layers
classifier.add(Conv2D(256, (3, 3), padding="same"))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())

# Max Pooling with Dropout 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# classifier.add(Dropout(0.2))

# First set of FC or Dense Layers
classifier.add(Flatten())
classifier.add(Dense(256))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())
# classifier.add(Dropout(0.5))

# Second set of FC or Dense Layers
classifier.add(Dense(256))
classifier.add(Activation('relu'))
# classifier.add(BatchNormalization())
# classifier.add(Dropout(0.5))

# Final Dense Layer
classifier.add(Dense(5))
classifier.add(Activation("softmax"))

print(classifier.summary())

In [ ]:
classifier.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.0001),
              metrics = ['accuracy'])

In [ ]:
                    
# # checkpoint = ModelCheckpoint("../input/simpsons_little_vgg.h5",
# #                              monitor="val_loss",
# #                              mode="min",
# #                              save_best_only = True,
# #                              verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

# reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
#                               factor = 0.2,
#                               patience = 3,
#                               verbose = 1,
#                               min_delta = 0.00001)

# # we put our call backs into a callback list
callbacks = [earlystop]

In [ ]:
history = classifier.fit_generator(generator=batches, steps_per_epoch=batches.n,
                             epochs=25,validation_data=val_batches,
                             validation_steps=val_batches.n, callbacks=callbacks)

In [ ]:
# best = save_best_model(10, "output", 5, ".hdf5")
# model.load_weights(best)
# model.save("best.h5s")

In [ ]:
X_test_.shape

In [ ]:


predprobs = classifier.predict_classes(X_test_)
#np.sum([[0, 1], [0, 5]], axis=0)



In [ ]:
# predprobs

In [ ]:
# predprobs

In [ ]:
predictions = []
for i in predprobs:
    predictions.append(i)

In [ ]:
# predictions

In [ ]:
np.unique(predictions)

In [ ]:
X_test_.shape, X_train_.shape

In [ ]:
# y_val

In [ ]:
id_code = df_test["id_code"].values.tolist()

In [ ]:
# for i in range(len(id_code)):
#     imgid = id_code[i]
#     newimgid = imgid[:-4]
#     id_code[i] = newimgid

In [ ]:


subfile = pd.DataFrame({"id_code":id_code, "diagnosis":predictions})



In [ ]:
subfile.to_csv('submission.csv',index=False)